In [ ]:
# -*- coding: utf-8 -*-
"""
Este script genera un dashboard interactivo de un modelo económico (DD-AA y Mercado Cambiario)
utilizando la librería Bokeh. El resultado es un único archivo HTML interactivo.
"""

# --- 1. Importaciones de Librerías ---
from bokeh.plotting import figure, show
from bokeh.io import output_file, save, output_notebook # Importar output_notebook
from bokeh.layouts import layout, column, row
from bokeh.models import ColumnDataSource, CustomJS, Slider, Select, HoverTool, Div, Arrow, VeeHead

import numpy as np

# # Configurar Bokeh para mostrar en el cuaderno - Descomentar si la visualización directa funciona
# output_notebook()

# --- 2. Definición de Parámetros Fijos del Modelo ---
RANGO_Y = np.linspace(0, 25, 100)
RANGO_I = np.linspace(0, 15, 100)
IP_INTERCEPT = 20
IP_PENDIENTE = 1.5
AA_INTERCEPT = 22
AA_PENDIENTE = 1.0
DD_INTERCEPT = 2
DD_PENDIENTE = 0.8

# --- 3. Cálculo del Equilibrio Inicial ---
y_eq1 = (AA_INTERCEPT - DD_INTERCEPT) / (DD_PENDIENTE + AA_PENDIENTE)
e_eq1 = DD_INTERCEPT + DD_PENDIENTE * y_eq1
i_eq1 = (IP_INTERCEPT - e_eq1) / IP_PENDIENTE

# --- 4. Creación de Fuentes de Datos (ColumnDataSource) ---
# Estas fuentes de datos contendrán toda la información que se actualizará dinámicamente.

# Fuentes para las curvas iniciales y las desplazadas
source_ip = ColumnDataSource(data=dict(x=RANGO_I, y=IP_INTERCEPT - IP_PENDIENTE * RANGO_I))
source_ip_nueva = ColumnDataSource(data=dict(x=[], y=[]))
source_dd = ColumnDataSource(data=dict(x=RANGO_Y, y=DD_INTERCEPT + DD_PENDIENTE * RANGO_Y))
source_dd_nueva = ColumnDataSource(data=dict(x=[], y=[]))
source_aa = ColumnDataSource(data=dict(x=RANGO_Y, y=AA_INTERCEPT - AA_PENDIENTE * RANGO_Y))
source_aa_nueva = ColumnDataSource(data=dict(x=[], y=[]))

# Fuentes para los puntos de equilibrio
source_eq1 = ColumnDataSource(data=dict(y=[y_eq1], e=[e_eq1], i=[i_eq1]))
source_eq2 = ColumnDataSource(data=dict(y=[], e=[], i=[]))

# Fuente para la flecha de ajuste en el mercado cambiario
source_flecha_mc = ColumnDataSource(data=dict(x_start=[], y_start=[], x_end=[], y_end=[]))

# Fuentes para las líneas de proyección de los equilibrios
source_eq1_lines_mc = ColumnDataSource(data=dict(
    x0=[0, i_eq1], y0=[e_eq1, 0], x1=[i_eq1, i_eq1], y1=[e_eq1, e_eq1]
))
source_eq1_lines_ddaa = ColumnDataSource(data=dict(
    x0=[0, y_eq1], y0=[e_eq1, 0], x1=[y_eq1, y_eq1], y1=[e_eq1, e_eq1]
))
source_eq2_lines_mc = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[]))
source_eq2_lines_ddaa = ColumnDataSource(data=dict(x0=[], y0=[], x1=[], y1=[]))

# --- 5. Creación de las Figuras (Gráficos) ---

# Herramienta Hover para mostrar coordenadas
hover_tool = HoverTool(tooltips=[("Producción (Y)", "@y{0.2f}"),
                                 ("Tipo de Cambio (e)", "@e{0.2f}"),
                                 ("Tasa de Interés (i)", "@i{0.2f}")])

# Gráfico 1: Mercado Cambiario
p_mc = figure(height=400, width=500, title="Mercado Cambiario",
              x_range=(0, 15), y_range=(-5, 30),
              x_axis_label="Tasa de Interés (i)", y_axis_label="Tipo de Cambio (e)",
              tools=[hover_tool, "pan,wheel_zoom,box_zoom,reset,save"])

p_mc.line('x', 'y', source=source_ip, line_width=2, color='darkblue', legend_label='IP (Retorno Depósitos Nacionales)')
p_mc.line('x', 'y', source=source_ip_nueva, line_width=2, color='darkblue', line_dash='dashed', legend_label="IP' (Nueva)")
p_mc.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq1_lines_mc, line_width=1, color='gray', line_dash='dashed')
p_mc.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq2_lines_mc, line_width=1, color='lightcoral', line_dash='dashed')
p_mc.circle(x='i', y='e', source=source_eq1, size=10, color='black', legend_label='Equilibrio Inicial (Eq-1)')
p_mc.circle(x='i', y='e', source=source_eq2, size=10, color='red', legend_label='Equilibrio Final (Eq-2)')
flecha_mc_renderer = Arrow(end=VeeHead(size=15, fill_color="darkmagenta"),
                           x_start='x_start', y_start='y_start', x_end='x_end', y_end='y_end',
                           source=source_flecha_mc, line_color="darkmagenta", line_width=2)
p_mc.add_layout(flecha_mc_renderer)


# Gráfico 2: Modelo DD-AA
p_ddaa = figure(height=400, width=500, title="Modelo DD-AA",
                x_range=(0, 25), y_range=(-5, 30),
                x_axis_label="Producción (Y)", y_axis_label="Tipo de Cambio (e)",
                tools=[hover_tool, "pan,wheel_zoom,box_zoom,reset,save"])

p_ddaa.line('x', 'y', source=source_dd, line_width=2, color='darkgreen', legend_label='Curva DD')
p_ddaa.line('x', 'y', source=source_dd_nueva, line_width=2, color='darkgreen', line_dash='dashed', legend_label="DD' (Nueva)")
p_ddaa.line('x', 'y', source=source_aa, line_width=2, color='crimson', legend_label='Curva AA')
p_ddaa.line('x', 'y', source=source_aa_nueva, line_width=2, color='crimson', line_dash='dashed', legend_label="AA' (Nueva)")
p_ddaa.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq1_lines_ddaa, line_width=1, color='gray', line_dash='dashed')
p_ddaa.segment(x0='x0', y0='y0', x1='x1', y1='y1', source=source_eq2_lines_ddaa, line_width=1, color='lightcoral', line_dash='dashed')
p_ddaa.circle(x='y', y='e', source=source_eq1, size=10, color='black', legend_label='Equilibrio Inicial (Eq-1)')
p_ddaa.circle(x='y', y='e', source=source_eq2, size=10, color='red', legend_label='Equilibrio Final (Eq-2)')

for p in [p_mc, p_ddaa]:
    p.legend.location = "top_right"
    p.legend.click_policy = "hide"
    p.title.align = 'center'
    p.title.text_font_size = '14pt'

# --- 6. Creación de Widgets Interactivos ---

# Widget para seleccionar el tipo de shock
shock_select = Select(title="Tipo de Shock:", value="Ninguno", options=['Ninguno', 'Fiscal', 'Monetario', 'Cambiario'])

# Widget para controlar la magnitud del shock
shock_slider = Slider(start=-5, end=5, value=0, step=0.5, title="Magnitud del Shock")

# Div para mostrar las explicaciones
explanation_div = Div(text="""
    <h3>Bienvenido al Dashboard del Modelo DD-AA</h3>
    <p>
    Selecciona un <b>tipo de shock</b> y ajusta su <b>magnitud</b> usando los controles de arriba
    para observar cómo se ajusta la economía a un nuevo equilibrio. Las explicaciones del mecanismo
    de ajuste aparecerán aquí.
    </p>
    """, width=1000, height=200)

# --- 7. Lógica de Interacción con CustomJS ---
# Este código JavaScript se ejecutará en el navegador cada vez que cambie un widget.

callback = CustomJS(args=dict(
    source_ip_n=source_ip_nueva,
    source_dd_n=source_dd_nueva,
    source_aa_n=source_aa_nueva,
    source_eq1=source_eq1,
    source_eq2=source_eq2,
    source_flecha=source_flecha_mc,
    source_eq2_lines_mc=source_eq2_lines_mc,
    source_eq2_lines_ddaa=source_eq2_lines_ddaa,
    shock_select=shock_select,
    shock_slider=shock_slider,
    explanation_div=explanation_div,
    RANGO_I=RANGO_I,
    RANGO_Y=RANGO_Y,
    params={
        'IP_I': IP_INTERCEPT, 'IP_P': IP_PENDIENTE,
        'AA_I': AA_INTERCEPT, 'AA_P': AA_PENDIENTE,
        'DD_I': DD_INTERCEPT, 'DD_P': DD_PENDIENTE
    }
), code="""
    // --- Obtener valores de los widgets y datos iniciales ---
    const shock_type = shock_select.value;
    const shock_value = shock_slider.value;

    const ip_i = params.IP_I; const ip_p = params.IP_P;
    const aa_i = params.AA_I; const aa_p = params.AA_P;
    const dd_i = params.DD_I; const dd_p = params.DD_P;

    const i_eq1 = source_eq1.data.i[0];
    const e_eq1 = source_eq1.data.e[0];

    // --- Reiniciar datos de las curvas nuevas y equilibrio final ---
    source_ip_n.data = { x: [], y: [] };
    source_dd_n.data = { x: [], y: [] };
    source_aa_n.data = { x: [], y: [] };
    source_eq2.data = { y: [], e: [], i: [] };
    source_flecha.data = { x_start: [], y_start: [], x_end: [], y_end: [] };
    source_eq2_lines_mc.data = { x0: [], y0: [], x1: [], y1: [] };
    source_eq2_lines_ddaa.data = { x0: [], y0: [], x1: [], y1: [] };

    let y_eq2, e_eq2, i_eq2;
    let titulo = "<h3>Análisis de Equilibrio</h3>";
    let cuerpo = "<p>Selecciona un tipo de shock para ver el mecanismo de ajuste.</p>";

    // --- Lógica para cada tipo de shock ---
    if (shock_type === 'Fiscal') {
        const nuevo_dd_i = dd_i + shock_value;
        y_eq2 = (aa_i - nuevo_dd_i) / (dd_p + aa_p);
        e_eq2 = nuevo_dd_i + dd_p * y_eq2;
        i_eq2 = (ip_i - e_eq2) / ip_p;

        source_dd_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_dd_i + dd_p * y) };
        source_flecha.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };

        if (shock_value > 0) {
            titulo = "<h3>Mecanismo de Ajuste: Política Fiscal Expansiva</h3>";
            cuerpo = `
                1. <b>Inicio</b>: Aumenta el gasto autónomo (G ↑).<br>
                2. <b>Mercado de Bienes</b>: La Demanda Agregada (DA) aumenta, desplazando la curva DD a la derecha.<br>
                3. <b>Mercado de Dinero</b>: El aumento en la producción (Y ↑) incrementa la demanda de dinero, elevando la tasa de interés (i ↑).<br>
                4. <b>Mercado Cambiario</b>: La mayor tasa de interés atrae capitales, apreciando el tipo de cambio (e ↓).<br>
                5. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más apreciado (e₁ < e₀).
            `;
        } else if (shock_value < 0) {
            titulo = "<h3>Mecanismo de Ajuste: Política Fiscal Contractiva</h3>";
            cuerpo = `
                1. <b>Inicio</b>: Disminuye el gasto autónomo (G ↓).<br>
                2. <b>Mercado de Bienes</b>: La Demanda Agregada (DA) disminuye, desplazando la curva DD a la izquierda.<br>
                3. <b>Mercado de Dinero</b>: La reducción en la producción (Y ↓) disminuye la demanda de dinero, reduciendo la tasa de interés (i ↓).<br>
                4. <b>Mercado Cambiario</b>: La menor tasa de interés provoca salida de capitales, depreciando el tipo de cambio (e ↑).<br>
                5. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más depreciado (e₁ > e₀).
            `;
        }

    } else if (shock_type === 'Monetario') {
        const nuevo_aa_i = aa_i + shock_value;
        y_eq2 = (nuevo_aa_i - dd_i) / (dd_p + aa_p);
        e_eq2 = dd_i + dd_p * y_eq2;
        i_eq2 = (ip_i - e_eq2) / ip_p;

        source_aa_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_aa_i - aa_p * y) };
        source_flecha.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };

        if (shock_value > 0) {
            titulo = "<h3>Mecanismo de Ajuste: Política Monetaria Expansiva</h3>";
            cuerpo = `
                1. <b>Inicio</b>: El banco central aumenta la oferta monetaria (Mˢ/P ↑).<br>
                2. <b>Mercado de Dinero</b>: Exceso de oferta de dinero, lo que reduce la tasa de interés (i ↓).<br>
                3. <b>Mercado Cambiario</b>: La menor tasa de interés provoca salida de capitales, depreciando el tipo de cambio (e ↑).<br>
                4. <b>Modelo DD-AA</b>: El aumento de Mˢ desplaza la curva AA hacia arriba.<br>
                5. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más depreciado (e₁ > e₀).
            `;
        } else if (shock_value < 0) {
            titulo = "<h3>Mecanismo de Ajuste: Política Monetaria Contractiva</h3>";
            cuerpo = `
                1. <b>Inicio</b>: El banco central reduce la oferta monetaria (Mˢ/P ↓).<br>
                2. <b>Mercado de Dinero</b>: Exceso de demanda de dinero, lo que eleva la tasa de interés (i ↑).<br>
                3. <b>Mercado Cambiario</b>: La mayor tasa de interés atrae capitales, apreciando el tipo de cambio (e ↓).<br>
                4. <b>Modelo DD-AA</b>: La reducción de Mˢ desplaza la curva AA hacia abajo.<br>
                5. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más apreciado (e₁ < e₀).
            `;
        }

    } else if (shock_type === 'Cambiario') {
        const nuevo_aa_i = aa_i + shock_value;
        const nuevo_ip_i = ip_i + shock_value;
        y_eq2 = (nuevo_aa_i - dd_i) / (dd_p + aa_p);
        e_eq2 = dd_i + dd_p * y_eq2;
        i_eq2 = (nuevo_ip_i - e_eq2) / ip_p;

        source_ip_n.data = { x: RANGO_I, y: RANGO_I.map(i => nuevo_ip_i - ip_p * i) };
        source_aa_n.data = { x: RANGO_Y, y: RANGO_Y.map(y => nuevo_aa_i - aa_p * y) };
        source_flecha.data = { x_start: [i_eq1], y_start: [e_eq1], x_end: [i_eq2], y_end: [e_eq2] };

        if (shock_value > 0) {
            titulo = "<h3>Mecanismo de Ajuste: Shock de Expectativas Cambiarias (Positivo)</h3>";
            cuerpo = `
                1. <b>Inicio</b>: Aumenta la expectativa del tipo de cambio futuro (Eᵉ ↑). Se espera una depreciación.<br>
                2. <b>Mercado Cambiario</b>: Aumenta el retorno esperado de depósitos extranjeros. Salida de capitales deprecia el tipo de cambio (e ↑).<br>
                3. <b>Curvas de Activos</b>: La curva IP y la curva AA se desplazan hacia arriba.<br>
                4. <b>Resultado Final</b>: Nuevo equilibrio con mayor producción (Y₁ > Y₀) y un tipo de cambio más depreciado (e₁ > e₀).
            `;
        } else if (shock_value < 0) {
            titulo = "<h3>Mecanismo de Ajuste: Shock de Expectativas Cambiarias (Negativo)</h3>";
            cuerpo = `
                1. <b>Inicio</b>: Disminuye la expectativa del tipo de cambio futuro (Eᵉ ↓). Se espera una apreciación.<br>
                2. <b>Mercado Cambiario</b>: Disminuye el retorno esperado de depósitos extranjeros. Entrada de capitales aprecia el tipo de cambio (e ↓).<br>
                3. <b>Curvas de Activos</b>: La curva IP y la curva AA se desplazan hacia abajo.<br>
                4. <b>Resultado Final</b>: Nuevo equilibrio con menor producción (Y₁ < Y₀) y un tipo de cambio más apreciado (e₁ < e₀).
            `;
        }
    }

    if (shock_type !== 'Ninguno') {
        source_eq2.data = { y: [y_eq2], e: [e_eq2], i: [i_eq2] };
        source_eq2_lines_mc.data = {
            x0: [0, i_eq2],
            y0: [e_eq2, 0],
            x1: [i_eq2, i_eq2],
            y1: [e_eq2, e_eq2]
        };
        source_eq2_lines_ddaa.data = {
            x0: [0, y_eq2],
            y0: [e_eq2, 0],
            x1: [y_eq2, y_eq2],
            y1: [e_eq2, e_eq2]
        };
    }

    // --- Actualizar el texto de explicación y emitir cambios ---
    explanation_div.text = titulo + "<p>" + cuerpo + "</p>";

    source_ip_n.change.emit();
    source_dd_n.change.emit();
    source_aa_n.change.emit();
    source_eq2.change.emit();
    source_flecha.change.emit();
    source_eq2_lines_mc.change.emit();
    source_eq2_lines_ddaa.change.emit();
"""
)

# Vincular el callback a los widgets
shock_select.js_on_change('value', callback)
shock_slider.js_on_change('value', callback)


# --- 8. Organizar el Layout del Dashboard ---
controles = column(shock_select, shock_slider)
graficos = row(p_mc, p_ddaa)
dashboard = layout([
    [controles],
    [graficos],
    [explanation_div]
])


# --- 9. Generar el Archivo HTML o mostrar en el cuaderno ---
output_file("dashboard_economico.html", title="Dashboard Modelo DD-AA") # Descomentar esta línea
save(dashboard) # Descomentar esta línea

# show(dashboard) # Mostrar directamente en el cuaderno - Comentar esta línea

print("¡Dashboard generado con éxito!")
print("Abre el archivo 'dashboard_economico.html' en tu navegador.")